In [2]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.nosdeputes.fr/16/dossier/1582"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [3]:
job_elements = soup.find("div", class_="seances_dossier").find_all("a")

for job_element in job_elements:
    print(job_element)
    print(job_element.get("href"))

<a href="/16/seance/849#table_1582">Séance en hémicycle du lundi 6 février 2023 à 16h00</a>
/16/seance/849#table_1582
<a href="/16/seance/850#table_1582">Séance en hémicycle du lundi 6 février 2023 à 21h30</a>
/16/seance/850#table_1582
<a href="/16/seance/852#table_1582">Séance en hémicycle du mardi 7 février 2023 à 15h00 (<span class="list_com">26 commentaires</span>)</a>
/16/seance/852#table_1582
<a href="/16/seance/853#table_1582">Séance en hémicycle du mardi 7 février 2023 à 21h30</a>
/16/seance/853#table_1582
<a href="/16/seance/854#table_1582">Séance en hémicycle du mercredi 8 février 2023 à 15h00</a>
/16/seance/854#table_1582
<a href="/16/seance/855#table_1582">Séance en hémicycle du mercredi 8 février 2023 à 21h30</a>
/16/seance/855#table_1582
<a href="/16/seance/859#table_1582">Séance en hémicycle du vendredi 10 février 2023 à 9h00</a>
/16/seance/859#table_1582
<a href="/16/seance/860#table_1582">Séance en hémicycle du vendredi 10 février 2023 à 15h00</a>
/16/seance/860#table_

In [4]:
tmp = job_elements[0].get("href")
print(tmp)

URL = "https://www.nosdeputes.fr"+tmp
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

/16/seance/849#table_1582


In [12]:
interventions = soup.find_all("div", class_="intervention")
folder_name = ""
for i in range(0, len(interventions)):
    inter = interventions[i]
    if inter.find("div", id="table_1582"):
        print(inter)
        folder_name = inter.find("h2", {'class': 'section'}).text
        interventions = interventions[i:]
        break

<div class="intervention" id="inter_10a149442c87d8cf530fabb407e884b7">
<div id="table_1582"><span class="source"><a href="#sommaire">Retour au sommaire</a> - <a href="#table_1582">Permalien</a></span><br/><h2 class="section"><a href="/16/dossier/1582">Projet de loi de financement rectificative de la sécurité sociale pour 2023</a></h2> </div></div>


In [13]:
for i in range(0, len(interventions)):
    inter = interventions[i]
    if not inter.find("div", {'class': 'intervenant'}):
        if inter.find("h2", {'class': 'section'}).text != folder_name:
            interventions = interventions[:i]
            break

In [32]:
dataframe = []

for inter in interventions:
    intervenant = inter.find("div", {'class': 'intervenant'})
    if intervenant and not intervenant.find("div", {'class': 'didascalie'}) \
    and intervenant.find('img', {'class': 'jstitle'}):
        text = intervenant.find('img', {'class': 'jstitle'})['title']
        name, group = text.split(' -- ')[0], text.split(' -- ')[1]
        group = group.strip('()')
        group_name, group_parlementaire = group.split(':')
        dataframe.append({
            "intervenant" : name,
            "parti" : group_parlementaire.strip(),
            "intervention" : intervenant.find("div", {'class': 'texte_intervention'}).text
        })
    

In [2]:
import requests
from bs4 import BeautifulSoup

URL_du_dossier = "https://www.nosdeputes.fr/16/dossier/1582"
page_dossier = requests.get(URL_du_dossier)

soup = BeautifulSoup(page_dossier.content, "html.parser")

sceances = soup.find("div", class_="seances_dossier").find_all("a")

dataframe = []

for sceance in sceances:
    URL_sceance = sceance.get("href")
    URL_sceance = "https://www.nosdeputes.fr"+URL_sceance
    page_sceance = requests.get(URL_sceance)

    soup = BeautifulSoup(page_sceance.content, "html.parser")

    #We get every intervention done in the sceance
    interventions = soup.find_all("div", class_="intervention")
    folder_name = ""

    #We filter the first lines of off topic discussions
    for i in range(0, len(interventions)):
        inter = interventions[i]
        if inter.find("div", id="table_1582"):
            folder_name = inter.find("h2", {'class': 'section'}).text
            interventions = interventions[i:]
            break
    
    #And the last lines of off topic discussions
    for i in range(0, len(interventions)):
        inter = interventions[i]
        if not inter.find("div", {'class': 'intervenant'}):
            if inter.find("h2", {'class': 'section'}).text != folder_name:
                interventions = interventions[:i]
                break
    
    #Now based on what remains we build our dataframe of interventions
    for inter in interventions:
        intervenant = inter.find("div", {'class': 'intervenant'})
        #We filter the interventions that are not from a person
        if intervenant and not intervenant.find("div", {'class': 'didascalie'}) \
        and intervenant.find('img', {'class': 'jstitle'}):
            text = intervenant.find('img', {'class': 'jstitle'})['title']
            name, group = text.split(' -- ')[0], text.split(' -- ')[1]
            group = group.strip('()')
            group_name, group_parlementaire = group.split(':')
            dataframe.append({
                "intervenant" : name,
                "parti" : group_parlementaire.strip(),
                "intervention" : intervenant.find("div", {'class': 'texte_intervention'}).text
            })

import pandas as pd
df = pd.DataFrame(dataframe)
df.head()

,intervenant,parti,intervention
0,Yaël Braun-Pivet,REN,\nL'ordre du jour appelle la discussion du pro...
1,Danièle Obono,LFI,"\nNon, pas question !"
2,Louis Boyard,LFI,"\nNon, ça ne va pas se passer comme ça ! Vous ..."
3,Yaël Braun-Pivet,REN,"\nLa parole est à M. le ministre du travail, d..."
4,André Chassaigne,GDR,\nSuspension ! La suspension est de droit !


In [3]:
df.shape

(7381, 3)

In [4]:
df.to_csv("tmp/interventions.csv", index=False)